In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf
from pyspark.sql.functions import size
from pyspark.sql.functions import explode
from pyspark.sql.functions import first
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import arrays_zip
from pyspark.sql.functions import min as pmin
from pyspark.sql.functions import max as pmax

In [3]:
from pyspark.sql.types import ArrayType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import VectorUDT

In [6]:
from pyspark.ml import Pipeline

In [7]:
from pyspark.mllib.stat import Statistics

In [8]:
import numpy as np

In [9]:
from pyspark.ml.pipeline import Transformer  
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.functions import size
from pyspark.sql.functions import array_union


class AsDenseTransformer(Transformer, HasInputCol, HasOutputCol):  
    def __init__(self, inputCol=None, outputCol=None, vecSize=None):
        super(AsDenseTransformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol
    
    def _transform(self, dataset):
        out_col = self.outputCol
        in_col = dataset[self.inputCol]

        asDense = udf(lambda s: DenseVector(s), VectorUDT()) 

        return dataset.withColumn(out_col, asDense(in_col))

In [10]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

In [11]:
spark = SparkSession.builder.getOrCreate()

Cargamos los datos

In [12]:
df = spark.read.csv('DataML_asym.csv', sep=',', inferSchema=True, header=True)

Se obtienen la cantidad maxima y minima de filas

In [13]:
df.show()

+---+------------------+------------------+------------------+-------------------+-------------------+---+---+---+------+
|VID|                X1|                X2|                X3|                 X4|                 X5| X6| X7|  Y|VLabel|
+---+------------------+------------------+------------------+-------------------+-------------------+---+---+---+------+
|  1|-0.960083983054105| -1.59895954297434| 0.308259853184722|   1.09736681973913|   1.69253595668614|0.0|0.0|  1|     1|
|  1| -1.46371949568538|-0.822724935536291|  2.10240861328983| -0.185861609258113|  -1.08617493676837|0.0|0.0|  1|     1|
|  1| 0.229556836998471|-0.273978626917343|-0.192478397717161|   1.60598813110087|   1.44571830505158|0.0|0.0|  1|     1|
|  1| -2.04041365142407|-0.369608074500547|  1.35634932099867|  0.639168616995513| -0.362054420080345|0.0|0.0|  1|     1|
|  1|-0.847930641437632|-0.209296614546657| 0.747613334230839|  0.275436229365676|  -1.60394691701022|0.0|0.0|  1|     1|
|  1| -1.59127902405805|

In [14]:
max_min_row = df.groupby('VID').count().agg(
    pmax('count').alias('max'),
    pmin('count').alias('min')
).collect()[0]

In [15]:
max_rows, min_rows = max_min_row['max'], max_min_row['min']

In [16]:
print(max_rows, min_rows)

1400 800


Se agrupan las columnas X1, X2, X3, X4, X5, X6, X7 a un vector

In [17]:
pad_fix_length = udf(
    lambda arr: arr + [0.0] * (max_rows - len(arr[:max_rows])), 
    ArrayType(DoubleType())
)

In [18]:
assembler = VectorAssembler(
    inputCols=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7'],
    outputCol="features")

df_vectorized = assembler.transform(df)

In [20]:
df_processed = df_vectorized.groupby('VID').agg(
    collect_list('features').alias('X'),
    first('Y').alias('Y'),
    first('VLabel').alias('VLabel')
).withColumn('size', size(col('X')))

In [22]:
df_grouped = df_vectorized.groupby('VID').agg(
    collect_list('X1').alias('X1'),
    collect_list('X2').alias('X2'),
    collect_list('X3').alias('X3'),
    collect_list('X4').alias('X4'),
    collect_list('X5').alias('X5'),
    collect_list('X6').alias('X6'),
    collect_list('X7').alias('X7'),
    first('Y').alias('Y'),
    first('VLabel').alias('VLabel')
).withColumn('size', size(col('X1')))

In [24]:
df_grouped.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+------+----+
| VID|                  X1|                  X2|                  X3|                  X4|                  X5|                  X6|                  X7|  Y|VLabel|size|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+------+----+
| 148|[0.39123361738572...|[-1.2888114223931...|[-0.6206498767479...|[-0.4460907837666...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|  1|     1| 888|
| 463|[0.78206005038687...|[-0.5445008176036...|[-0.8413289851357...|[-0.7867371379827...|[0.93496187586504...|[0.56139814762433...|[-0.2066235277046...|  1|     1|1388|
| 471|[-1.0179853213517...|[-0.8904868694508...|[1.7915364692595,...|[-2.1014650757005...|[-2.0915941730321...|[0.20263986126343...|[-0.5439198420522.

In [25]:
df_padded = df_grouped.withColumn(
    'X1', pad_fix_length(col('X1'))
).withColumn(
    'X2', pad_fix_length(col('X2'))
).withColumn(
    'X3', pad_fix_length(col('X3'))
).withColumn(
    'X4', pad_fix_length(col('X4'))
).withColumn(
    'X5', pad_fix_length(col('X5'))
).withColumn(
    'X6', pad_fix_length(col('X6'))
).withColumn(
    'X7', pad_fix_length(col('X7'))
)

In [101]:
pipeline_min = Pipeline(stages=[
    AsDenseTransformer(inputCol='X1', outputCol='X1'),
    AsDenseTransformer(inputCol='X2', outputCol='X2'),
    AsDenseTransformer(inputCol='X3', outputCol='X3'),
    AsDenseTransformer(inputCol='X4', outputCol='X4'),
    AsDenseTransformer(inputCol='X5', outputCol='X5'),
    AsDenseTransformer(inputCol='X6', outputCol='X6'),
    AsDenseTransformer(inputCol='X7', outputCol='X7'),
    PCA(k=min_rows, inputCol="X1", outputCol="pcaX1"),
    PCA(k=min_rows, inputCol="X2", outputCol="pcaX2"),
    PCA(k=min_rows, inputCol="X3", outputCol="pcaX3"),
    PCA(k=min_rows, inputCol="X4", outputCol="pcaX4"),
    PCA(k=min_rows, inputCol="X5", outputCol="pcaX5"),
    PCA(k=min_rows, inputCol="X6", outputCol="pcaX6"),
    PCA(k=min_rows, inputCol="X7", outputCol="pcaX7"),
])

In [102]:
model_min = pipeline_min.fit(df_padded)

In [103]:
df_padded_processed_min = model_min.transform(df_padded)

In [ ]:
a = AsDenseTransformer(inputCol='X1', outputCol='X1')
b = AsDenseTransformer(inputCol='X1', outputCol='X1')
c = AsDenseTransformer(inputCol='X1', outputCol='X1')

In [104]:
df_padded_processed_min_exploded = df_padded_processed_min.withColumn(
    'pcaX1', to_array('pcaX1')
).withColumn(
    'pcaX2', to_array('pcaX2')
).withColumn(
    'pcaX3', to_array('pcaX3')
).withColumn(
    'pcaX4', to_array('pcaX4')
).withColumn(
    'pcaX5', to_array('pcaX5')
).withColumn(
    'pcaX6', to_array('pcaX6')
).withColumn(
    'pcaX7', to_array('pcaX7')
).withColumn(
    'tmp', arrays_zip('pcaX1', 'pcaX2', 'pcaX3', 'pcaX4', 'pcaX5', 'pcaX6', 'pcaX7')
).withColumn(
    "tmp", explode("tmp")
).select(
    'VID',
    col("tmp.pcaX1").alias('X1'),
    col("tmp.pcaX2").alias('X2'),
    col("tmp.pcaX3").alias('X3'),
    col("tmp.pcaX4").alias('X4'),
    col("tmp.pcaX5").alias('X5'),
    col("tmp.pcaX6").alias('X6'),
    col("tmp.pcaX7").alias('X7'),
    'Y',
    'VLabel'
)

In [105]:
df_padded_processed_min_exploded.write.parquet('pca_minimo_800')

In [21]:
df_padded.withColumn(
    'tmp', arrays_zip('X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7')
).withColumn(
    "tmp", explode("tmp")
).select(
    'VID',
    col("tmp.X1").alias('X1'),
    col("tmp.X2").alias('X2'),
    col("tmp.X3").alias('X3'),
    col("tmp.X4").alias('X4'),
    col("tmp.X5").alias('X5'),
    col("tmp.X6").alias('X6'),
    col("tmp.X7").alias('X7'),
    'Y',
    'VLabel'
).write.parquet('padded_maximo_1400')

In [22]:
sizes = df_padded.select('size').collect()

In [25]:
sizes = list(map(lambda x: x['size'], sizes))

In [56]:
average = int(np.round(np.average(sizes)))

In [57]:
truncate_arr = udf(
    lambda arr: arr[:average], 
    ArrayType(DoubleType())
)

In [58]:
df_below_avg = df_padded.where(col('size') <= average)
df_over_avg = df_padded.where(col('size') > average)

In [59]:
df_below_avg_truncated = df_below_avg.withColumn(
    'X1', truncate_arr(col('X1'))
).withColumn(
    'X2', truncate_arr(col('X2'))
).withColumn(
    'X3', truncate_arr(col('X3'))
).withColumn(
    'X4', truncate_arr(col('X4'))
).withColumn(
    'X5', truncate_arr(col('X5'))
).withColumn(
    'X6', truncate_arr(col('X6'))
).withColumn(
    'X7', truncate_arr(col('X7'))
).withColumn(
    'tmp', arrays_zip('X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7')
).withColumn(
    "tmp", explode("tmp")
).select(
    'VID',
    col("tmp.X1").alias('X1'),
    col("tmp.X2").alias('X2'),
    col("tmp.X3").alias('X3'),
    col("tmp.X4").alias('X4'),
    col("tmp.X5").alias('X5'),
    col("tmp.X6").alias('X6'),
    col("tmp.X7").alias('X7'),
    'Y',
    'VLabel'
)

In [34]:
pipeline_avg = Pipeline(stages=[
    AsDenseTransformer(inputCol='X1', outputCol='X1'),
    AsDenseTransformer(inputCol='X2', outputCol='X2'),
    AsDenseTransformer(inputCol='X3', outputCol='X3'),
    AsDenseTransformer(inputCol='X4', outputCol='X4'),
    AsDenseTransformer(inputCol='X5', outputCol='X5'),
    AsDenseTransformer(inputCol='X6', outputCol='X6'),
    AsDenseTransformer(inputCol='X7', outputCol='X7'),
    PCA(k=average, inputCol="X1", outputCol="pcaX1"),
    PCA(k=average, inputCol="X2", outputCol="pcaX2"),
    PCA(k=average, inputCol="X3", outputCol="pcaX3"),
    PCA(k=average, inputCol="X4", outputCol="pcaX4"),
    PCA(k=average, inputCol="X5", outputCol="pcaX5"),
    PCA(k=average, inputCol="X6", outputCol="pcaX6"),
    PCA(k=average, inputCol="X7", outputCol="pcaX7"),
])

In [36]:
model_avg = pipeline_avg.fit(df_over_avg)

In [37]:
df_over_avg_processed = model_avg.transform(df_over_avg)

In [38]:
df_over_avg_processed_exploded = df_over_avg_processed.withColumn(
    'pcaX1', to_array('pcaX1')
).withColumn(
    'pcaX2', to_array('pcaX2')
).withColumn(
    'pcaX3', to_array('pcaX3')
).withColumn(
    'pcaX4', to_array('pcaX4')
).withColumn(
    'pcaX5', to_array('pcaX5')
).withColumn(
    'pcaX6', to_array('pcaX6')
).withColumn(
    'pcaX7', to_array('pcaX7')
).withColumn(
    'tmp', arrays_zip('pcaX1', 'pcaX2', 'pcaX3', 'pcaX4', 'pcaX5', 'pcaX6', 'pcaX7')
).withColumn(
    "tmp", explode("tmp")
).select(
    'VID',
    col("tmp.pcaX1").alias('X1'),
    col("tmp.pcaX2").alias('X2'),
    col("tmp.pcaX3").alias('X3'),
    col("tmp.pcaX4").alias('X4'),
    col("tmp.pcaX5").alias('X5'),
    col("tmp.pcaX6").alias('X6'),
    col("tmp.pcaX7").alias('X7'),
    'Y',
    'VLabel'
)

In [54]:
df_average_processed = df_below_avg_truncated.select(
    'VID', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Y', 'VLabel'
).union(
    df_over_avg_processed_exploded.select(
        'VID', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Y', 'VLabel'
    )
)

In [66]:
df_average_processed.write.parquet('pca_padded_avg_1099')